# Basics of mobile robotics [MICRO-452]  
## Project report, EPFL Robotic Master, 12.12.21

<img src="image/logo-epfl.png" alt="Drawing" style="width: 200px;"/>

Author: **Nour Tnani [296442], Xavier Nal [sciper], Alicia Mauroux [274618], Antoine Perrin [sciper]**

## 0. Introduction
Our project is to find a parking spot to our Thymio and to guide it to this spot using Global Navigation. 
Thymio will evolve on a map where there is two houses and four parking spots. Three of this spots are occupied and only one of them is free. There will be some "dumb" Thymio that will evolve in this map too, our Thymio have to avoid them, using Local Navigation. 

Here is our map:
Green = parking spots, Blue = obstacles, Black = "dumb" Thymio's path
<img src="image/map.jpg" alt="Drawing" style="width: 600px;"/>


## 1. Initialization
### Connection to Thymio and libraries
The next cells import all the libraries and the other folders that we will need

In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

CancelledError: 

In [2]:
import cv2

In [3]:
import math

import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async

import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 
from Mouvement import*
from vision import* 
from optimal_path import* 
from KalmanFilter import KalmanFilter



ModuleNotFoundError: No module named 'asgiref'

### Variables and constants


In [4]:
#Initialize the initial grid and position
#start , goal , grid = "init_map"
#start_pos = (0,0)
#goal = (43,33)
map_lenght = 5
nb_of_square_by_side = 50
current = 1

#variables
global motor_left_target, motor_right_target



## 2. Useful functions
Functions in order to use Thymio's actuators and sensors easily

In [5]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed

In [6]:
@tdmclient.notebook.sync_var
def proxi():
    """
    Returns the proximity values of the Thymio 
    """
    global prox_horizontal
    return prox_horizontal

## 3. Current goal update
To be completed by Antoine

Next cell is to create an instance Robot and Map 
  

In [ ]:
global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)
kalman_bool = False
#pourcentage_reduc = 1

In [7]:
George = Robot()
Lausanne = Map(map_lenght, nb_of_square_by_side)
KF=KalmanFilter(0.1, [0, 0])

In [ ]:
#VideoCap = vision_initialization()
VideoCap=cv2.VideoCapture(0)

print('Hello World')


Init part
Global control 

--> Optimal path calculation

In [4]:
# Prend une première image
ret, frame=VideoCap.read()

# Set the pourcentage value between the nb of pixel and nb of square by side
Lausanne.set_pourcentage(frame)

# Set the robot goal, position and angle
goal = init_goal(frame, Lausanne.get_pourcentage())
pos_robot, angle = update(frame, Lausanne.get_pourcentage(), kalman_bool)  # fonction qui retourne la position angle etc....
George.set_angle(angle)
George.set_goal(goal)
George.set_start_pos(pos_robot)
George.set_pos(pos_robot)
print ("goal", goal)
print ("position", )


mask = mask_map_init(frame)
cv2.imshow("mask", mask)

Lausanne.init_grid(mask)

# affiche la grille
cv2.imshow("final secure grid", Lausanne.get_map())



######
# Compute the optimal path and visited nodes
path, visitedNodes = op.path_computation(George.get_start() , George.get_goal() , Lausanne.get_lenght(), Lausanne.get_map())

George.set_path(path) 
George.set_visit_nodes(visitedNodes) 

# Display the optimal path
op.display_map(Lausanne.get_lenght(),  Lausanne.get_map(),  George.get_visit_nodes(), George.get_path(), George.get_start(), George.get_goal())

1

## 4. Vision
To be completed by Xavier

## 5. To follow the logical path
To be completed by Nour

## 6. Local Navigation
To be completed by me

## 7. Main loop

In [7]:

kalman_bool = False


print('while start')
while True:#(George.get_pos() != George.get_goal()):
    
    # Read the camera
    ret, frame=VideoCap.read()
    
    # Reupere la position et angle --> XAV
    #x_robot, y_robot,angle_robot = update(frame, Lausanne.get_pourcentage(), kalman_bool)
    pos_robot,angle_robot = update(frame, Lausanne.get_pourcentage())
    #KF.kalman_update(pos, speed)
    George.set_pos(pos_robot)
    George.set_angle(angle_robot)
    print ('position',pos_robot ,angle_robot)
    
    
    # Current goal update --> ANTOINE
    
    current = path_update(George.get_pos(), George.get_err_pos(), George.get_path(), George.get_current())
    
    # Check obstacle --> ALICIA
    
    # Faire avancer --> NOUR
    
    
    proximity = proxi()
    
        #LOCAL NAVIGATION
    if move:
        #Thymio is avoiding obstacles
        speed_l, speed_r, move = avoid_obstacle(prox_horizonta=proximity) 
    else:
        #Thymio is following the optimal path
        speed_l, speed_r = move_to_position(x_robot, y_robot, angle_robot, 152,262)
        #Thymio is checking if there's an obstacle in front of it
        move = check_cars(prox_horizonta=proximity)
        print (speed_l, speed_r)
        
    motors(speed_l, speed_r) 
    
    
    # Quit when we press q
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        break
     
    sleep(0.1)

print("The End")
    


Hello World
while start
position 0 0 0
0 0
debut 0 0 0
dist_debut 302.89932320822373
v 605.7986464164475
w 41.803931990020935
speed, 41 -41
41 -41
position 0 0 0
0 0
debut 0 0 0
dist_debut 302.89932320822373
v 605.7986464164475
w 41.803931990020935
speed, 41 -41
41 -41


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 1: invalid continuation byte

In [17]:
motors(0,0)

## 8. "Dumb" Thymio
We have implemented this code in another Thymio. In this part, the Thymio "dumb" will follow a line until it sees an object in front of it. In this case, it stops until the object is removed. 


To connect to the "dumb" Thymio:

In [ ]:
import tdmclient.notebook
await tdmclient.notebook.start()